In [1]:
# UI for reading ini config file -- initial stage
# Updated : 15 Oct
# Cliff : Only show userful columns 22nd Oct
import tkinter as tk
from tkinter import ttk
import configparser
from tkinter import messagebox

# Load INI file and read sections
# Chnage obj
def load_ini_file(ini_file):
    config = configparser.ConfigParser()
    config.read(ini_file)
    return config

# Display Ini selection in combobox.
def display_ini_section(event):
    selected_study = combobox.get()
    if selected_study:
        study_data = config[selected_study]
        details = "\n".join([f"{key} = {value}" for key, value in study_data.items()])
        text_area.delete("1.0", tk.END)
        text_area.insert(tk.INSERT, details)

# Validation for SequenceStartDate (8 numeric characters)
def restrict_chars(P):
    if len(P) > 8:
        return False
    return P.isdigit() or P == ""

# Validation for StudyRun (6 alphanumeric characters)
def validate_study_run(P):
    if len(P) > 7:
        return False
    return P.isalnum() or P == ""

# Submit code
def submit_values():
    selected_study = combobox.get()
    study_run_value = study_run_entry.get()
    sequence_start_value = sequence_start_entry.get()
    new_rate_files_value = new_rate_files_combobox.get()
    new_plans_value = new_plans_combobox.get()

    # Check if input values meet requirements
    if selected_study and study_run_value and sequence_start_value and new_rate_files_value and new_plans_value:
        if len(study_run_value) == 6 and study_run_value.isalnum() and len(sequence_start_value) == 8 and sequence_start_value.isdigit():
            config[selected_study]['StudyRun'] = study_run_value
            config[selected_study]['SequenceStartDate'] = sequence_start_value
            config[selected_study]['NewRateFiles'] = new_rate_files_value
            config[selected_study]['NewPlans'] = new_plans_value

            # Write config back to file
            with open(ini_file, 'w') as configfile:
                config.write(configfile)

            display_ini_section(None)
        else:
            messagebox.showerror("Invalid Input", "Ensure that StudyRun is 6 alphanumeric characters and SequenceStartDate is 8 numeric characters.")
    else:
       # messagebox.showerror("Missing Input", "All fields are required.",font=('Arial',12))
        messagebox.showerror("Missing Input", "All fields are required.")
    
    

# Main window starts here..
root = tk.Tk()
root.title("Measure Processing UI - 2.0")
root.geometry("600x700")

ini_file = "config_master.ini"
config = load_ini_file(ini_file)

# Labels & textboxes for inputs :
tk.Label(root, text="Select Study", font=('Arial', 12), fg="black").pack(pady=10)

# Main combobox
combobox = ttk.Combobox(root, values=config.sections(), width=30)
combobox.pack(pady=10)
combobox.bind("<<ComboboxSelected>>", display_ini_section)

# Text area
text_area = tk.Text(root, width=60, height=15)
text_area.pack(pady=20)

# Labels and Textboxes for StudyRun and SequenceStartDate inputs
frame_inputs = tk.Frame(root)
frame_inputs.pack(pady=10)

# StudyRun
tk.Label(frame_inputs, text="StudyRun:", font=('Arial', 10)).grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
vcmd_study_run = (root.register(validate_study_run), '%P')
study_run_entry = tk.Entry(frame_inputs, width=30, validate="key", validatecommand=vcmd_study_run)
study_run_entry.grid(row=0, column=1, padx=5, pady=5)

# SequenceStartDate
tk.Label(frame_inputs, text="SequenceStartDate:", font=('Arial', 10)).grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
vcmd_sequence_start = (root.register(restrict_chars), '%P')
sequence_start_entry = tk.Entry(frame_inputs, width=30, validate="key", validatecommand=vcmd_sequence_start)
sequence_start_entry.grid(row=1, column=1, padx=5, pady=5)

# New Rate Files
tk.Label(frame_inputs, text="New Rate Files:", font=('Arial', 10)).grid(row=2, column=0, padx=5, pady=5, sticky=tk.E)
new_rate_files_combobox = ttk.Combobox(frame_inputs, values=["True", "False"], width=28)
new_rate_files_combobox.grid(row=2, column=1, padx=5, pady=5)

# New Plans
tk.Label(frame_inputs, text="New Plans:", font=('Arial', 10)).grid(row=3, column=0, padx=5, pady=5, sticky=tk.E)
new_plans_combobox = ttk.Combobox(frame_inputs, values=["True", "False"], width=28)
new_plans_combobox.grid(row=3, column=1, padx=5, pady=5)

tk.Label(frame_inputs, text="Value of 1st input:", font=('Arial', 10)).grid(row=4, column=0, padx=5, pady=5, sticky=tk.E)


# Submit Button
submit_button = tk.Button(root, text="Submit", font=('Arial', 14), command=submit_values)
submit_button.pack(pady=20)

root.mainloop()
